config.py에 튜토리얼에 이용하였던 텍스트 파일의 주소가 있습니다. 각자의 실습에 맞도록 config.py를 만들어 주세요. 

이 튜토리얼에서 이용한 파일은 2016-10-20 하루의 뉴스를 수집한 30,091건의 뉴스기사 입니다. 

토크나이저는 KoNLPy의 Twitter를 이용하였습니다. 

In [2]:
from math import log
from pprint import pprint 
from soynlp.utils import DoublespaceLineCorpus
from config import twitter_tokenized_corpus

sents = DoublespaceLineCorpus(twitter_tokenized_corpus, iter_sent=True)

## Word frequency distribution

In [3]:
from collections import Counter
word_counter = Counter((word for sent in sents for word in sent.split()))

In [4]:
for min_count in [1, 2, 5, 10, 20, 30, 50, 100, 200, 500, 1000]:
    filtered_counter = list(filter(lambda x:x[1] >= min_count, word_counter.items()))
    
    print('min count = {}, num words = {}'.format(min_count, len(filtered_counter)))

min count = 1, num words = 76069
min count = 2, num words = 56738
min count = 5, num words = 38546
min count = 10, num words = 28063
min count = 20, num words = 19731
min count = 30, num words = 15572
min count = 50, num words = 11358
min count = 100, num words = 7067
min count = 200, num words = 4263
min count = 500, num words = 1951
min count = 1000, num words = 1038


등장 빈도가 작을수록 고유명사인 경우가 많습니다. 반대로 등장빈도가 높을수록 단어의 종류는 줄어들고, 남아있는 단어들은 다양한 상황에서 자주 쓰이는 단어들입니다. 

In [5]:
for min_count in [1, 2, 5, 10, 20, 30, 50, 100, 200, 500, 1000]:
    
    filtered_counter = list(filter(lambda x:x[1] >= min_count, word_counter.items()))
    least_top5 = sorted(filtered_counter, key=lambda x:x[1])[:5]
    least_top5 = [word for word, _ in least_top5]
    
    print('\n## min count = {}, num words = {}'.format(min_count, len(filtered_counter)))
    print(least_top5)


## min count = 1, num words = 76069
['화나면/Verb', '변기수/Noun', '안그/Noun', '입법권/Noun', '과잉행동/Noun']

## min count = 2, num words = 56738
['훑어보기/Noun', '최영수/Noun', '세달/Noun', '백승준/Noun', '겁박/Noun']

## min count = 5, num words = 38546
['발언권/Noun', '8160/Number', '분리하는/Verb', '후기성도/Noun', '인디씬/Noun']

## min count = 10, num words = 28063
['교황/Noun', '들여다볼/Verb', '제곱미터/Noun', '탄도탄/Noun', '다우지수/Noun']

## min count = 20, num words = 19731
['느린/Adjective', '생겨난/Verb', '윤지혜/Noun', '가벼/Adjective', '앓아/Verb']

## min count = 30, num words = 15572
['5500/Number', '지적하며/Verb', '옥천/Noun', '238/Number', '이오/Noun']

## min count = 50, num words = 11358
['신념/Noun', '동북아/Noun', '두루/Noun', '믿지/Verb', '수순/Noun']

## min count = 100, num words = 7067
['소액/Noun', '유지되/Verb', '바이어/Noun', '세월/Noun', '아이파크/Noun']

## min count = 200, num words = 4263
['전무/Noun', '고통/Noun', '유포/Noun', '실/PreEomi', '거든요/Eomi']

## min count = 500, num words = 1951
['강북/Noun', '오픈/Noun', '승객/Noun', '어/Noun', '벗어/Verb']

## min c

### Zipf's Law

Zipf's Law는 단어 빈도수와 단어 종류 사이에는 통계적으로 유의미한 법칙이 있다는 발견입니다. 흔히 단어의 log(num_words) = a $\times$ log(최소빈도수) + b 처럼 표현할 수 있다고 말합니다. 

많은 자연계의 현상들처럼 최소빈도수가 낮을수록 정말 다양한 단어들이 존재하지만, 최소빈도수의 한계가 100, 1000, 2000 처럼 높아질수록 그보다 더 많이 등장한 단어의 종류는 소수가 됩니다. 파레토 법칙이 단어 빈도의 분포에서도 존재합니다. 

머신러닝 알고리즘의 입장에서는 고유한 단어 하나 마다 하나의 feature에 해당됩니다. 그렇기 때문에 term frequency vector를 만들거나 Word2Vec과 같은 word embedding을 학습할 때, min count filtering을 수행합니다. 한 두 번 등장한 단어를 위하여 매우 큰 학습비용을 지불할 이유가 없을 뿐더러, overfitting의 위험이 있기 때문입니다. 

In [7]:
x = []
y = []

for min_count in [1, 2, 5, 10, 20, 30, 50, 100, 200, 500, 1000]:
    filtered_counter = list(filter(lambda x:x[1] >= min_count, word_counter.items()))
    x.append(log(min_count))
    y.append(log(len(filtered_counter)))
    
print('log(min_count)\tlog(num_words)\t= sum')
for xi, yi in zip(x, y):
    print('\t%.2f\t%.2f\t\t= %.2f' % (xi, yi, xi+yi))

log(min_count)	log(num_words)	= sum
	0.00	11.24		= 11.24
	0.69	10.95		= 11.64
	1.61	10.56		= 12.17
	2.30	10.24		= 12.54
	3.00	9.89		= 12.89
	3.40	9.65		= 13.05
	3.91	9.34		= 13.25
	4.61	8.86		= 13.47
	5.30	8.36		= 13.66
	6.21	7.58		= 13.79
	6.91	6.95		= 13.85


## n-gram

가장 간단한 방법은 n-gram이 몇 번 등장하였는지 카운팅 하는 것입니다. 

### Exraction method: Counting

In [9]:
def counting(max_n, min_count):
    ngram_counter = {}
    for sent in sents:
        words = tuple(sent.split()) # for saving memory
        for begin in range(len(words)-1): 
            for end in range(begin+2, min(begin+max_n, len(words))+1):
                ngram = words[begin:end]
                ngram_counter[ngram] = ngram_counter.get(ngram, 0) + 1

    ngram_counter = {ngram:count for ngram, count in ngram_counter.items() if count >= min_count}
    return ngram_counter

ngram_counter = counting(max_n = 3, min_count = 10)
print('num ngrams = {}'.format(len(ngram_counter)))

num ngrams = 190268


In [10]:
def check_ngram(word, position, topk=5):
    filtered_ngrams = filter(lambda x: x[0][position][:len(word)] == word, ngram_counter.items())
    filtered_ngrams = sorted(filtered_ngrams, key=lambda x:-x[1])
    pprint(filtered_ngrams[:topk])

In [11]:
check_ngram('영화', -1)

[(('열린/Verb', '영화/Noun'), 476),
 (('에서/Noun', '열린/Verb', '영화/Noun'), 288),
 (('코미디/Noun', '영화/Noun'), 204),
 (('국제/Noun', '영화제/Noun'), 200),
 (('에서/Josa', '열린/Verb', '영화/Noun'), 185)]


In [12]:
check_ngram('공연', 0)

[(('공연/Noun', '을/Josa'), 422),
 (('공연/Noun', '이/Josa'), 100),
 (('공연/Noun', '은/Josa'), 89),
 (('공연/Noun', '을/Josa', '펼/Verb'), 80),
 (('공연/Noun', '과/Josa'), 70)]


In [13]:
check_ngram('하다', -1)

[(('이기도/Josa', '하다/Verb'), 219),
 (('듯/Noun', '하다/Verb'), 91),
 (('을/Josa', '하다/Verb'), 72),
 (('를/Josa', '하다/Verb'), 70),
 (('기만/Eomi', '하다/Verb'), 53)]


뉴스 데이터이기 때문에 "무단전재 및 재배포 금지"가 trigram의 최빈어들로 등장하였습니다. KoNLPy.Twitter는 형태소분석을 하였기 때문에 복합명사인 "무단전재", "재배포"가 각각 "무단 + 전제"와 "재 + 배포"로 나누어졌습니다. 후처리를 통하여 이들을 하나의 명사로 묶을 수도 있습니다. 이 이야기는 tokenizer part에서 다루겠습니다. 

In [14]:
def check_trigram(topk=10):
    trigrams = filter(lambda x:len(x[0]) == 3, ngram_counter.items())
    trigrams = sorted(trigrams, key=lambda x:-x[1])[:topk]
    pprint(trigrams)

check_trigram()

[(('재/Noun', '배포/Noun', '금지/Noun'), 20436),
 (('및/Noun', '재/Noun', '배포/Noun'), 14687),
 (('전재/Noun', '및/Noun', '재/Noun'), 14340),
 (('무단/Noun', '전재/Noun', '및/Noun'), 14340),
 (('무단/Noun', '전재/Noun', '재/Noun'), 5178),
 (('전재/Noun', '재/Noun', '배포/Noun'), 5178),
 (('2016/Number', '10/Number', '20/Number'), 5169),
 (('고/Noun', '말했/Verb', '다/Eomi'), 5135),
 (('공감/Noun', '언론/Noun', '뉴시스/Noun'), 4274),
 (('20/Number', '일/Noun', '오후/Noun'), 4157)]


### Extracting method: bigram (Mikolov)

- [Word2Vec paper][word2vec_nips] $^{[1]}$에서 저자들은 mutual information과도 비슷한 간단한 bigram 점수 방법을 제안하였습니다. 

$score(w_i, w_j) = \frac{count(w_i, w_j) - \delta}{count(w_i) \times count(w_j)}$

이 점수는 최소한 $\delta$ 보다 많이 등장한 bigram $w_i - w_j$에 대하여 각 단어의 unigram 빈도수로 나누어 줍니다. 두 단어가 연속으로 등장한 횟수가 많더라도 각각의 단어가 흔한 단어인 경우, 이를 보정하는 의미입니다. 

\[1\]: [Mikolov, T., Sutskever, I., Chen, K., Corrado, G. S., & Dean, J. (2013). Distributed representations of words and phrases and their compositionality. In Advances in neural information processing systems][word2vec_nips]

[word2vec_nips]: https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality

In [15]:
def bigram_mikolov(delta):
    bigram_mikolov_scores = {}
    for ngram, count in ngram_counter.items():
        if not (len(ngram) == 2) or count <= delta:
            continue
        score = (count - delta) / (word_counter.get(ngram[0]) * word_counter.get(ngram[1]))
        bigram_mikolov_scores[ngram] = score
    return bigram_mikolov_scores

scoring 만 했을 뿐인데도 '무라카미 - 하루키'와 같이 의미있는 단어들이 bigram으로 추출됨을 볼 수 있습니다. 하지만 높은 점수를 보이는 bigram의 빈도수는 delta와 비슷합니다. 

Point Mutual Information (PMI)에서도 빈도수가 낮은 단어들로 구성된 ngram이 유독 높은 점수를 보입니다. 위 방법은 사실 PMI의 상수배와 같기 때문에, 동일한 문제가 발생합니다.

In [16]:
bigram_mikolov_scores = bigram_mikolov(delta=10)

for ngram, score in sorted(bigram_mikolov_scores.items(), key=lambda x:-x[1])[:20]:
    print('ngram = {} / score = {:.4} / count = {}'.format(ngram, score, ngram_counter[ngram]))

ngram = ('허심/Noun', '탄회/Noun') / score = 0.024 / count = 25
ngram = ('무라카미/Noun', '하루키/Noun') / score = 0.02381 / count = 21
ngram = ('로웰/Noun', '패독/Noun') / score = 0.02381 / count = 21
ngram = ('천호진/Noun', '장현성/Noun') / score = 0.02368 / count = 19
ngram = ('네시/Noun', '삼십삼분/Noun') / score = 0.02273 / count = 20
ngram = ('콰트로/Noun', '포르테/Noun') / score = 0.02257 / count = 23
ngram = ('복음서/Noun', '언해/Noun') / score = 0.02257 / count = 23
ngram = ('레프/Noun', '탈산/Noun') / score = 0.02256 / count = 19
ngram = ('장현성/Noun', '이시언/Noun') / score = 0.02222 / count = 18
ngram = ('악바리/Noun', '근성/Noun') / score = 0.02183 / count = 21
ngram = ('외유/Noun', '내강/Noun') / score = 0.02093 / count = 23
ngram = ('미안/Noun', '보협/Noun') / score = 0.02083 / count = 16
ngram = ('박재근/Noun', '엘리스/Noun') / score = 0.02083 / count = 15
ngram = ('꽁/Adverb', '냥/Noun') / score = 0.02071 / count = 24
ngram = ('해무리/Noun', '굽완/Noun') / score = 0.02069 / count = 25
ngram = ('피아트/Noun', '크라이슬러/Noun') / score = 0.02043 / c

delta를 좀 더 크게해도 경향은 비슷합니다. Zipf's Law 에서 배울 수 있던 것처럼, 빈도수는 상대적입니다. 하지만 delta가 어느 정도 커도 '족보 - 꼬인'이나 '의기 - 투합' 같은 phrase가 잘 추출됨을 볼 수 있습니다. 

하지만 '브렉시트'나 '프로듀스101'이 각각 '브렉 + 시트', '프로 + 듀스 + 101'로 나뉘어졌으며, '차세대'의 경우, '차세/Verb + 대/Eomi'로 나뉘어졌습니다. 이는 학습데이터나 단어 사전에 존재하지 않은 단어를 마주할 때 토크나이저 / 품사판별기가 자주 겪는 미등록단어 문제 (out-of-vocabulary, OOV) 입니다. 이 이야기는 토크나이저 / 품사판별 부분에서 좀 더 다루겠습니다. 

In [17]:
bigram_mikolov_scores = bigram_mikolov(delta=100)

for ngram, score in sorted(bigram_mikolov_scores.items(), key=lambda x:-x[1])[:20]:
    print('ngram = {} / score = {:.4} / count = {}'.format(ngram, score, ngram_counter[ngram]))

ngram = ('의기/Noun', '투합/Noun') / score = 0.002381 / count = 166
ngram = ('398/Number', '4441/Number') / score = 0.00223 / count = 199
ngram = ('몬스타/Noun', '엑스/Noun') / score = 0.002033 / count = 205
ngram = ('781/Number', '9711/Number') / score = 0.00198 / count = 358
ngram = ('세일/Noun', '페스/Noun') / score = 0.00196 / count = 166
ngram = ('차세/Verb', '대/Eomi') / score = 0.001929 / count = 274
ngram = ('가습기/Noun', '살균제/Noun') / score = 0.001864 / count = 222
ngram = ('브렉/Noun', '시트/Noun') / score = 0.001817 / count = 149
ngram = ('자유로이/Adverb', '접근할/Verb') / score = 0.001799 / count = 358
ngram = ('새판/Noun', '짜기/Verb') / score = 0.001775 / count = 161
ngram = ('비선/Noun', '실세/Noun') / score = 0.001707 / count = 263
ngram = ('애슈/Noun', '턴/Noun') / score = 0.001697 / count = 130
ngram = ('족보/Noun', '꼬인/Adjective') / score = 0.001641 / count = 445
ngram = ('반론/Noun', '정정/Noun') / score = 0.001639 / count = 358
ngram = ('체육관/Noun', '광부/Noun') / score = 0.001633 / count = 286
ngram = ('선/Verb'

### Extracting method: Extending Mikolov's method for trigram

앞서 bigram을 추출하기 위한 간단한 식을 살펴보았습니다. 

$score(w_i, w_j) = \frac{count(w_i, w_j) - \delta}{count(w_i) \times count(w_j)}$

trigram을 추출하기 위해서는 이를 위한 점수가 정의되어야 합니다. Mutual information을 2개에서 n개로 확장하는 방법은 다양하며, 절대적인 정의는 없습니다. 우리는 위의 식을 간단히 확장하여 trigram을 추출하는 식을 만들어 봅니다. 

아래의 식은 분모를 두 개의 bigram frequency로 나누었습니다. unigram은 문맥이 제대로 정의되지 않기 때문에 흔한 (여러 문맥에서 등장하는) 단어가 포함된 n-gram의 점수를 크게 낮출 수 있습니다. 이를 방지하기 위하여 bigram을 이용하도록 합니다. 

$score(w_i, w_j, w_k) = \frac{count(w_i, w_j, w_k) - \delta}{count(w_i, w_j) \times count(w_j, w_k)}$


In [18]:
def trigram_extending_mikolov(delta):
    trigram_scores = {}
    for ngram, count in ngram_counter.items():
        if not (len(ngram) == 3) or count <= delta:
            continue
        score = (count - delta) / (ngram_counter.get(ngram[:2]) * ngram_counter.get(ngram[1:]))
        trigram_scores[ngram] = score
    return trigram_scores

그 결과 '선녀-바위-해안'과 같은 trigram의 고유명사도 추출되지만, '을/Josa + 구형/Noun + 했/Verb'와 같은 어절이 나뉘어진 phrase도 추출됩니다. 

In [19]:
trigram_scores = trigram_extending_mikolov(delta=10)

for ngram, score in sorted(trigram_scores.items(), key=lambda x:-x[1])[:20]:
    print('ngram = {} / score = {:.4} / count = {}'.format(ngram, score, ngram_counter[ngram]))

ngram = ('알/Noun', '무/Noun', '바락/Noun') / score = 0.025 / count = 20
ngram = ('의왕/Noun', '백/Suffix', '운/Noun') / score = 0.025 / count = 20
ngram = ('양윤영/Noun', '박희현/Noun', '강소영/Noun') / score = 0.025 / count = 20
ngram = ('유조선/Noun', '해상/Noun', '충돌/Noun') / score = 0.025 / count = 20
ngram = ('법도/Noun', '통제/Noun', '불가능한/Adjective') / score = 0.025 / count = 20
ngram = ('시리즈/Noun', '런칭/Noun', '기념/Noun') / score = 0.025 / count = 20
ngram = ('멤버십/Noun', '기획/Noun', '사업/Noun') / score = 0.025 / count = 20
ngram = ('박희현/Noun', '강소영/Noun', '등/Noun') / score = 0.025 / count = 20
ngram = ('다/Josa', '가도/Noun', '촬영/Noun') / score = 0.025 / count = 20
ngram = ('이/Josa', '그대/Noun', '없/Adjective') / score = 0.025 / count = 20
ngram = ('한국/Noun', '총괄/Noun', '모바일/Noun') / score = 0.025 / count = 20
ngram = ('페레그린/Noun', '과/Josa', '이상한/Adjective') / score = 0.025 / count = 20
ngram = ('업자/Noun', '기타/Noun', '소득/Noun') / score = 0.025 / count = 20
ngram = ('게/Eomi', '새집/Noun', '다오/Noun') / score = 0.02

delta를 10에서 500으로 키워도 동일한 문제와 특징이 보여집니다. 

In [20]:
trigram_scores = trigram_extending_mikolov(delta=500)

for ngram, score in sorted(trigram_scores.items(), key=lambda x:-x[1])[:20]:
    print('ngram = {} / score = {:.4} / count = {}'.format(ngram, score, ngram_counter[ngram]))

ngram = ('보는/Verb', '창/Noun', '경제/Noun') / score = 0.0004821 / count = 841
ngram = ('질투/Noun', '의/Josa', '화신/Noun') / score = 0.0004808 / count = 880
ngram = ('미디어/Noun', '이데일리/Noun', '무단/Noun') / score = 0.0004588 / count = 777
ngram = ('정보/Noun', '미디어/Noun', '이데일리/Noun') / score = 0.0004588 / count = 777
ngram = ('제보/Noun', '및/Noun', '보도자료/Noun') / score = 0.0004582 / count = 777
ngram = ('경제/Noun', '정보/Noun', '미디어/Noun') / score = 0.0004547 / count = 777
ngram = ('종합/Noun', '경제/Noun', '정보/Noun') / score = 0.0004547 / count = 777
ngram = ('포즈/Noun', '를/Josa', '취하고/Verb') / score = 0.0004518 / count = 1035
ngram = ('를/Josa', '취하고/Verb', '있다/Adjective') / score = 0.0004463 / count = 957
ngram = ('리얼타임/Noun', '뉴스/Noun', '머니투데이/Noun') / score = 0.0004438 / count = 749
ngram = ('보이는/Verb', '리얼타임/Noun', '뉴스/Noun') / score = 0.0004438 / count = 749
ngram = ('뉴스/Noun', '머니투데이/Noun', '무단/Noun') / score = 0.0004438 / count = 749
ngram = ('이/Josa', '보이는/Verb', '리얼타임/Noun') / score = 0.0004334 /

## Filtering

우리가 기대하는 n-gram은 명사로 시작하는 phrase와 같은 것들입니다. Mutual information과 같은 통계 정보만을 이용할 경우에는 '-의/Josa + 작품/Noun'와 같이 실제로 유의미한 phrase가 아님에도 통계적으로는 매우 연관성이 높은 phrase가 존재합니다. 

기대하는 n-gram을 추출하기 위해서는 언어학적 정보를 이용해야 좋습니다. 필요에 따라 filter를 잘 만들어서 기대하는 형태의 ngram을 추출하면 됩니다. 그 결과 '가장 - 확실한 - 방법'과 같은 구문들이 상위권으로 올라왔습니다. 

In [21]:
def trigram_filter(scores):
    scores_ = {}
    for ngram, score in scores.items():
        if not ('/Noun' in ngram[0]):
            continue
        if '/Josa' in ngram[1]:
            continue
        if ('Eomi' in ngram[2]) or ('Number' in ngram[2]):
            continue
        scores_[ngram] = score
    return scores_

In [23]:
trigram_scores = trigram_extending_mikolov(delta=500)
trigram_scores = trigram_filter(trigram_scores)

for ngram, score in sorted(trigram_scores.items(), key=lambda x:-x[1])[:20]:
    print('ngram = {} / score = {:.4} / count = {}'.format(ngram, score, ngram_counter[ngram]))

ngram = ('미디어/Noun', '이데일리/Noun', '무단/Noun') / score = 0.0004588 / count = 777
ngram = ('정보/Noun', '미디어/Noun', '이데일리/Noun') / score = 0.0004588 / count = 777
ngram = ('제보/Noun', '및/Noun', '보도자료/Noun') / score = 0.0004582 / count = 777
ngram = ('경제/Noun', '정보/Noun', '미디어/Noun') / score = 0.0004547 / count = 777
ngram = ('종합/Noun', '경제/Noun', '정보/Noun') / score = 0.0004547 / count = 777
ngram = ('뉴스/Noun', '머니투데이/Noun', '무단/Noun') / score = 0.0004438 / count = 749
ngram = ('리얼타임/Noun', '뉴스/Noun', '머니투데이/Noun') / score = 0.0004438 / count = 749
ngram = ('영상/Noun', '제보/Noun', '받/Verb') / score = 0.000426 / count = 1625
ngram = ('뉴스/Noun', '가치나/Noun', '화제/Noun') / score = 0.000426 / count = 1625
ngram = ('사진/Noun', '또는/Adverb', '영상/Noun') / score = 0.000426 / count = 1625
ngram = ('가치나/Noun', '화제/Noun', '성/Suffix') / score = 0.0004239 / count = 1625
ngram = ('판단/Noun', '되는/Verb', '사진/Noun') / score = 0.0004229 / count = 1625
ngram = ('독자/Noun', '여러분/Noun', '의/Josa') / score = 0.0004212 / co

## scikit-learn CounterVectorizer (ngram)

scikit-learn에서 제공하는 CounterVectorizer에서도 ngram을 자동으로 추출하여 term frequency matrix를 만들어줍니다. 

하지만 이 때에는 품사를 판단하여 filtering을 한다거나, score를 계산하여 최소한의 빈도수 (delta)나 score threshold를 두는 것은 아닙니다. 그렇기 때문에 빠른 예비분석 (pilot)을 위한 방법으로는 CounterVectorizer의 ngram 기능을 쓰더라도, 제대로 된 ngram 을 이용한 분석을 위해서는 CounterVectorizer의 tokenizer를 튜닝해서 사용하기 바랍니다. 

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

min_n, max_n = 1, 3
my_tokenizer = lambda sent:['{}/{}'.format(word,pos) for word, pos in twitter.pos(sent) if pos == 'Noun']

vectorizer = CountVectorizer(tokenizer=my_tokenizer,
                             ngram_range=(min_n, max_n))